In [1]:
import sqlite3
import datetime as dt
import pandas as pd

class Database:

    def __init__(self, db_name):
        self.db_name = db_name
        self.connection = sqlite3.connect(f'{self.db_name}')
        self.cursor = self.connection.cursor()

    def create_table(
            self,
            table_name: str,
            table_fields: dict,
    ):
        fields_str = ", ".join(
            [f'{field_name} {field_type}' 
            for field_name, field_type in table_fields.items()]
        )
        query = f'CREATE TABLE [IF NOT EXISTS] {table_name}({fields_str})'
        
        try:
            while self.connection:
                self.cursor.execute(query)
                self.connection.commit()
        except sqlite3.OperationalError as error:
            if str(error).startswith(f'table {table_name} already exists'):
                raise sqlite3.OperationalError(f'Table {table_name} already exists')
            else:
                raise error

    def insert_data(
            self,
            destination_table: str,
            data: 'pd.DataFrame'
    ):
        fields = ", ".join(data.columns)
        values = [tuple(row) for row in data.values] 
        query = f'INSERT INTO {destination_table}({fields}) VALUES ({", ".join(["?"]*len(data.columns))})'

        while self.connection:
            self.cursor.executemany(query, values)
            self.connection.commit()

    def delete_data(
            self,
            destination_table: str,
            data: 'pd.DataFrame'
    ):
        fields = ", ".join(data.columns)
        values = [tuple(row) for row in data.values] 
        query = f'DELETE FROM {destination_table}({fields}) VALUES ({", ".join(["?"]*len(data.columns))})'

        while self.connection:
            self.cursor.executemany(query, values)
            self.connection.commit()

    def select_columns(
            self,
            columns: list,
            source: str = 'sqlite_master',
            order: str = '',
    ):
        if order == '':
            items_list = ", ".join(columns)
            query = f'SELECT {items_list} FROM {source}'
        else:
            items_list = ", ".join(columns)
            query = f'SELECT {items_list} FROM {source} ORDER BY {order}'
        
        return self.cursor.execute(query).fetchall()

    def close_connection(self):
        self.connection.close()

In [ ]:
categories_table = 'categories'

categories_table_fields = {
    'id': 'INTEGER PRIMARY KEY',
    'category': 'TEXT'
}

categories_data = [
    (1, 'Commedy'),
    (2, 'Tragedy')
]

categories_data_df = pd.DataFrame(categories_data, columns=list(categories_table_fields.keys()))

In [ ]:
movies_table = 'movies'

movies_table_fields = {
    'id': 'INTEGER PRIMARY KEY',
    'title': 'TEXT',
    'category': 'TEXT',
    'year': 'DATE',
    'score': 'FLOAT'
}

movies_data = [
    (1, 'Monty Python and the Holy Grail', 'Commedy', dt.date(1975, 1, 1), 8.2),
    (2, 'And Now for Something Completely Different', 'Tragedy', dt.date(1980, 1, 1), 7.5),
    (3, 'Terminator', 'Commedy', dt.date(1975, 1, 1), 10.0),
    (4, 'Titanic', 'Tragedy', dt.date(1980, 1, 1), 6.5)
]

movies_data_df = pd.DataFrame(movies_data, columns=list(movies_table_fields.keys()))

In [ ]:
sql_tools = Database('tutorial.db')

In [ ]:
sql_tools.create_table(
    table_name=movies_table,
    table_fields=movies_table_fields)

sql_tools.create_table(
    table_name=categories_table,
    table_fields=categories_table_fields)

In [ ]:
sql_tools.insert_data(
    destination_table=movies_table,
    data=movies_data_df
)

sql_tools.insert_data(
    destination_table=categories_table,
    data=categories_data_df
)


In [ ]:
sql_tools.select_columns(
    columns=['id', 'title'],
    source=movies_table,
)

In [ ]:
sql_tools.close_connection()

In [ ]:
sql_tools.cursor.execute('VACUUM')